In [2]:
import pandas as pd
df=pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df=pd.get_dummies(df,drop_first=True,dtype=int)
df.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


In [4]:
X=df.drop('HeartDisease',axis=1)
y=df['HeartDisease']

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_scaled=sc.fit_transform(X)
X_scaled[:3]

array([[-1.4331398 ,  0.41090889,  0.82507026, -0.55134134,  1.38292822,
        -0.83243239,  0.51595242,  2.07517671, -0.53283777, -0.22967867,
         0.81427482, -0.49044933, -0.8235563 , -1.00218103,  1.15067399],
       [-0.47848359,  1.49175234, -0.17196105, -0.55134134,  0.75415714,
         0.10566353, -1.93816322, -0.48188667,  1.87674385, -0.22967867,
         0.81427482, -0.49044933, -0.8235563 ,  0.99782372, -0.86905588],
       [-1.75135854, -0.12951283,  0.7701878 , -0.55134134, -1.52513802,
        -0.83243239,  0.51595242,  2.07517671, -0.53283777, -0.22967867,
        -1.22808661,  2.03894663, -0.8235563 , -1.00218103,  1.15067399]])

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
scores=cross_val_score(SVC(),X_scaled,y,cv=5)
scores.mean()

np.float64(0.8256593014967926)

In [8]:
from sklearn.ensemble import BaggingClassifier
bag_model=BaggingClassifier(estimator=SVC(),n_estimators=100,max_samples=0.8,random_state=10)
scores=cross_val_score(bag_model,X_scaled,y,cv=5)
scores.mean()

np.float64(0.82892017106201)

In [9]:
from sklearn.tree import DecisionTreeClassifier
scores=cross_val_score(DecisionTreeClassifier(),X_scaled,y,cv=5)
scores.mean()

np.float64(0.7253920171062009)

In [10]:
bag_model=BaggingClassifier(estimator=DecisionTreeClassifier(),n_estimators=100,max_samples=0.8,random_state=10)
scores=cross_val_score(bag_model,X_scaled,y,cv=5)
scores.mean()

np.float64(0.79727963886909)

Comparing the performance of SVM and Decision Tree classifiers, it makes the most sense to use bagging with Decision Trees.
This is because Decision Trees are high-variance models, and bagging is an ensemble technique that effectively reduces variance by averaging predictions from multiple base learners.
In your experiment, bagging improved the Decision Tree accuracy significantly from 72% to 79%, a 7% increase.
In contrast, the SVM classifier, which is already a low-variance model, only saw a marginal improvement from 82.56% to 82.89%.
Therefore, bagging is more beneficial when used with high-variance models like Decision Trees, rather than stable models like SVM.
